In [178]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn import linear_model
%matplotlib inline

In [179]:
crime = pd.read_csv("https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv"
                 )

In [180]:
#cleaning, removing header rows
crime.columns = crime.iloc[3]
crime.droprow = crime.drop(crime.index[0:4], inplace=True)

#remove na values from population
crime.dropna(subset = ['Population'], inplace=True)
#remove , and conver to float (for desired columns)
crime = crime.replace(',','',regex=True)
crime.iloc[:,1:] = crime.iloc[:,1:].apply(lambda x: pd.to_numeric(x))
crime = crime.reset_index(drop=True)

In [181]:
crime.describe()

3,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
count,3.480000e+02,348.000000,348.000000,0.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,187.000000
mean,4.003763e+04,201.594828,1.566092,NaN,5.864943,72.902299,121.261494,792.606322,119.683908,637.017241,35.905172,1.871658
std,4.500374e+05,2815.268504,18.303673,NaN,60.425452,1031.032873,1706.131730,7659.724746,924.948789,6346.054451,403.423826,10.693411
min,5.260000e+02,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.003000e+03,2.000000,0.000000,NaN,0.000000,0.000000,1.000000,40.500000,6.000000,31.000000,0.000000,0.000000
50%,7.233500e+03,6.000000,0.000000,NaN,0.000000,1.000000,4.000000,112.500000,17.500000,94.000000,2.000000,0.000000
75%,1.842750e+04,22.000000,0.000000,NaN,2.000000,5.000000,14.000000,341.000000,51.250000,287.250000,7.000000,1.000000
max,8.396126e+06,52384.000000,335.000000,NaN,1112.000000,19170.000000,31767.000000,141971.000000,16606.000000,117931.000000,7434.000000,132.000000


In [182]:
#Looks like there is an outlier here (its NYC), in all categories. Let's find out which city it is and drop
crime['Population'].idxmax() 

216

In [183]:
crime.iloc[216]['City']

'New York'

In [184]:
crime = crime.drop(216)

In [185]:
#now keeping only desired columns
crime_regr = crime.loc[:, ['City', 'Property\ncrime','Population', 'Murder and\nnonnegligent\nmanslaughter', 'Robbery']]
#rename
crime_regr.columns = ['City','Property_crime','Population', 'Murder', 'Robbery']
#reset index
crime_regr = crime_regr.reset_index(drop=True)

In [186]:
crime_regr.describe()

,Property_crime,Population,Murder,Robbery
count,347.000000,347.000000,347.000000,347.000000
mean,385.752161,15956.685879,0.605187,17.867435
std,1034.369072,27080.218837,3.707090,94.972492
min,0.000000,526.000000,0.000000,0.000000
25%,40.000000,2997.000000,0.000000,0.000000
50%,112.000000,7187.000000,0.000000,1.000000
75%,340.500000,18160.500000,0.000000,5.000000
max,12491.000000,258789.000000,47.000000,1322.000000


In [187]:
crime_regr.head(20)

,City,Property_crime,Population,Murder,Robbery
0,Adams Village,12,1861,0,0
1,Addison Town and Village,24,2577,0,0
2,Akron Village,16,2846,0,0
3,Albany,4090,97956,8,227
4,Albion Village,223,6388,0,4
5,Alfred Village,46,4089,0,3
6,Allegany Village,10,1781,0,0
7,Amherst Town,2118,118296,1,31
8,Amityville Village,210,9519,0,4
9,Amsterdam,405,18182,0,12


In [189]:
#giving murder and robbery indicators and creating pop ^2
crime_regr.Robbery = crime_regr.Robbery.where(crime_regr.Robbery ==0,1)
crime_regr.Murder = crime_regr.Murder.where(crime_regr.Murder ==0,1)
crime_regr['Pop^2'] =crime_regr['Population'] **2

In [190]:
from sklearn import linear_model
regr = linear_model.LinearRegression()

regr.fit(crime_regr[['Population','Pop^2','Murder','Robbery']],crime_regr['Property_crime'])


array([  1.29510871e-02,   1.14920528e-07,   1.95038666e+02,
         9.28205845e+01])

In [191]:
print(regr.coef_)
print(regr.intercept_)

[  1.29510871e-02   1.14920528e-07   1.95038666e+02   9.28205845e+01]
-16.8158663217
